In [2]:
!pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/e0/7a/08f0ea19a0c835e88aad011083d9dda69a9dfa4585c3453b3bd842eb7bed/selenium-4.21.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/76/51/12d78ec8abcbda51d8f115d98ebd3ee3da9d9d9af00ac69d3097c5b8d51a/trio-0.25.1-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions>=4.9.0 from https://files.pythonhosted.org/packages/e1/4d/d612de852a0bc64a64418e1cef25fe1914c5b1611e34cc271ed7e36174c8/typing_extensions-4.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for attrs>=23.2.0 from https://files.pythonhosted.org/packages/e0/44/827b2a91a5816512fcaf3cc4ebc465ccd5d598c45cefa670

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.6.4 requires visions[type_image_path]==0.7.5, but you have visions 0.7.4 which is incompatible.


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from urllib.parse import urlparse, parse_qs, urlencode, urlunparse


import time
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\casti\OneDrive\Documents\A Springboard\Capstone 3 - Investments\data\interim\city_urls.csv")
df.head()

,url
0,https://www.airbnb.com/s/newyork-ny/homes
1,https://www.airbnb.com/s/losangeles-ca/homes
2,https://www.airbnb.com/s/chicago-il/homes
3,https://www.airbnb.com/s/dallas-tx/homes
4,https://www.airbnb.com/s/houston-tx/homes


In [4]:
url_list = list(df.url)
url_list[:5]

['https://www.airbnb.com/s/newyork-ny/homes',
 'https://www.airbnb.com/s/losangeles-ca/homes',
 'https://www.airbnb.com/s/chicago-il/homes',
 'https://www.airbnb.com/s/dallas-tx/homes',
 'https://www.airbnb.com/s/houston-tx/homes']

In [22]:
def insert_reviews_path(city_url):
    """Add the string '/reviews' to the appropriate place in an Airbnb listing URL. 
    The new URL navigates to the reviews page of that listing."""
    parsed_url = urlparse(city_url)
    query_params = parse_qs(parsed_url.query)
    new_path = parsed_url.path + "/reviews"
    query_params_str = urlencode(query_params, doseq=True)
    new_url = urlunparse((parsed_url.scheme, parsed_url.netloc, new_path, parsed_url.params, query_params_str, parsed_url.fragment))
    return new_url

def collect_review_urls(city_url, max_pages=5):
    """ 
    Initiates a Selenium webdriver in Chrome that opens an Airbnb URL of listings in a certain city;
    navigates to each listing on the page (20 total) and collects URLs of specific listings;
    uses insert_reviews_path function defined above to transform listing URLs into review page URLs;
    stores and returns these review page URLs as 'review_urls' list.
    """
    listings_class = "itu7ddv"
    driver = webdriver.Chrome()
    driver.get(city_url)

    review_urls = []
    current_page = 1
    
    print(f"Total pages to scan: {max_pages}.")
    
    while current_page <= max_pages:
        WebDriverWait(driver, 60).until(
            EC.visibility_of_all_elements_located((By.CLASS_NAME, listings_class)))
        
        listings = driver.find_elements(By.CLASS_NAME, listings_class)
        
        status = 'Not ready - detecting more listings. . .'
        
        while status == 'Not ready - detecting more listings. . .':
            if len(listings) < 20:
                status = 'Not ready - detecting more listings. . .'
                WebDriverWait(driver, 60).until(
                    EC.visibility_of_all_elements_located((By.CLASS_NAME, listings_class)))
                listings = driver.find_elements(By.CLASS_NAME, listings_class)
                print(status)
                
            else:
                status = f"Ready! {len(listings)} total listings detected on page {current_page}."
                print(status)
                
                if len(listings) > 20:
                    print(f"Estimated {len(listings) - 20} are not valid elements - expect hiccups.")
                    
        for listing in listings:
            try:
                driver.execute_script("arguments[0].scrollIntoView();", listing)
                driver.execute_script("arguments[0].click();", listing)
                
                original_window = driver.current_window_handle
                WebDriverWait(driver, 20).until(EC.number_of_windows_to_be(2))
                handles = driver.window_handles
                driver.switch_to.window(handles[-1])
                
                WebDriverWait(driver, 10).until(EC.url_contains("airbnb.com/rooms/"))
                
                current_url = driver.current_url
                review_url = insert_reviews_path(current_url)
                review_urls.append(review_url)

                driver.close()
                driver.switch_to.window(original_window)
                
            except Exception as e:
                print(f'Could not access listing: {listing}. Exception: {e}')
                continue
        
        # Move to the next page
        if current_page < max_pages:
            try:
                next_page_xpath = "//*[@aria-label='Next']"
                next_button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, next_page_xpath)))
                driver.execute_script("arguments[0].scrollIntoView();", next_button)
                next_button.click()
                print(f"Moving to page {current_page + 1} . . .")
                time.sleep(2)
            except Exception as e:
                print(f'Could not navigate to next page: {e}')
                break
        
        current_page += 1
    
    driver.quit()
    return review_urls


In [23]:
def review_scraper(review_url):
    driver = webdriver.Chrome()
    review_data = []
    
    try:
        driver.get(review_url)

        review_class = "r1are2x1"

        WebDriverWait(driver, 60).until(
            EC.visibility_of_all_elements_located((By.CLASS_NAME, review_class)))

        reviews_captured = 0
        max_reviews = 200
        

        while reviews_captured < max_reviews:

            reviews = driver.find_elements(By.CLASS_NAME, review_class)
            current_review_count = len(reviews)

            if current_review_count <= reviews_captured:
                break  # No new reviews are being loaded

            for review in reviews[reviews_captured:]:
                if reviews_captured >= max_reviews:
                    break

                driver.execute_script("arguments[0].scrollIntoView();", review)

                try:
                    # Extracting the relevant data from each review element
                    name = review.find_element(By.CLASS_NAME, "hpipapi").text
                    rating_info = review.find_element(By.CLASS_NAME, "s78n3tv").text
                    review_text = review.find_element(By.CLASS_NAME, "r1bctolv").text

                    # Collecting the data
                    review_data.append({
                        "name": name,
                        "rating_info": rating_info,
                        "review_text": review_text
                    })

                    reviews_captured += 1

                except Exception as e:
                    print(f'Could not extract review data: {e}')
                    continue

            # Scroll down to load more reviews if needed
            if reviews_captured < max_reviews:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                WebDriverWait(driver, 20).until(
                    EC.visibility_of_all_elements_located((By.CLASS_NAME, review_class)))

    except Exception as e:
        print(f"An error occurred while scraping reviews from {review_url}: {e}")
        # Log the error or take other appropriate actions
    
    finally:
        driver.quit()
        
    return review_data


In [24]:
def extract_city_from_url(url):
    # Parse the URL
    parsed_url = urlparse(url)
    
    # Extract the city from the path
    # The city is the first segment after '/s/' in the path
    path_segments = parsed_url.path.split('/')
    
    # Ensure the path structure is as expected
    if len(path_segments) >= 3 and path_segments[1] == 's':
        city = path_segments[2]
        return city
    else:
        raise ValueError("URL structure is not as expected")

In [25]:
def scrapeBNBrevs(city_url):
    
    city_name = extract_city_from_url(city_url)
    
    print(f"Collecting Review URLs for {city_name}. . .")
    
    review_urls = set(collect_review_urls(city_url, max_pages=5))
        
    reviews = []
    max_reviews = 2000
    
    print(f"Capturing reviews for {city_name}. . .")
    for idx, url in enumerate(review_urls):
        
        if len(reviews) >= max_reviews:
            break
        
        print(f"Scanning {idx / 100} potential listings.")
        reviews.extend(review_scraper(url))
        print(f'Updated review count for {city_name}: {len(reviews)} / {max_reviews} max.')

        
    # Filter out duplicate reviews
        
    seen = set()
    unique_reviews = [d for d in reviews if tuple(d.items()) not in seen and not seen.add(tuple(d.items()))]
    print(f'Total unique reviews for {city_name}: {len(unique_reviews)}.')    
    
    review_df = pd.DataFrame(unique_reviews, columns=unique_reviews[0].keys())
    
    return review_df


In [20]:
# Iterate over each city URL
for idx, city_url in enumerate(url_list[-2:]) :

    city_name = extract_city_from_url(city_url)
    
    print('*'*50)
    print({city_name}, {idx} / {len(url(list))} total cities.)
    print('*'*50)
    
    # Scrape reviews for the current city
    city_reviews_df = scrapeBNBrevs(city_url)
    
    # Save the DataFrame with a unique name based on the city name
    filename = r"C:\\Users\\casti\\OneDrive\\Documents\\A Springboard\\Capstone 3 - Investments\\data\\interim\\city_reviews\\{}_reviews.csv".format(city_name)
    city_reviews_df.to_csv(filename, index=False)
    print('*'*50)
    print(f"Saved {city_name} reviews to file.")
    print('*'*50)

Ready! 20 total listings detected on page 1.
**************************************************
Moving to page 2 . . .
**************************************************
Ready! 20 total listings detected on page 2.
**************************************************
Moving to page 3 . . .
**************************************************
Ready! 22 total listings detected on page 3.
Could not access listing: <selenium.webdriver.remote.webelement.WebElement (session="bb544684bd8b0e77be237bef3917efdd", element="f.E6BB9FDF17CB0E67A5F2318E23A03C0F.d.5A2D1F9757045D8187F0F359946E18B5.e.613")>. Exception: Message: 

Could not access listing: <selenium.webdriver.remote.webelement.WebElement (session="bb544684bd8b0e77be237bef3917efdd", element="f.E6BB9FDF17CB0E67A5F2318E23A03C0F.d.5A2D1F9757045D8187F0F359946E18B5.e.618")>. Exception: Message: 

**************************************************
Moving to page 4 . . .
**************************************************
Ready! 21 total listings det

Review 15 captured.
Review 16 captured.
Review 17 captured.
Review 18 captured.
Review 19 captured.
Review 20 captured.
Review 21 captured.
Review 22 captured.
Review 23 captured.
Review 24 captured.
Review 25 captured.
Review 26 captured.
Review 27 captured.
Review 28 captured.
Review 29 captured.
Review 30 captured.
Review 31 captured.
Review 32 captured.
Review 33 captured.
Review 34 captured.
Review 35 captured.
Review 36 captured.
Review 37 captured.
Review 38 captured.
Review 39 captured.
Review 40 captured.
Review 41 captured.
Review 42 captured.
Review 43 captured.
Review 44 captured.
Review 45 captured.
Review 46 captured.
Review 47 captured.
Review 48 captured.
Review 49 captured.
Review 50 captured.
Review 51 captured.
Review 52 captured.
Review 53 captured.
Review 54 captured.
Review 55 captured.
Review 56 captured.
Review 57 captured.
Review 58 captured.
Review 59 captured.
Review 60 captured.
Review 61 captured.
Review 62 captured.
Review 63 captured.
Review 64 captured.


Review 7 captured.
Review 8 captured.
Review 9 captured.
Review 10 captured.
Review 11 captured.
Review 12 captured.
Review 13 captured.
Review 14 captured.
Review 15 captured.
Review 16 captured.
Review 17 captured.
Review 18 captured.
Review 19 captured.
Review 20 captured.
Review 21 captured.
Review 22 captured.
Review 23 captured.
Review 24 captured.
Review 25 captured.
Review 26 captured.
Review 27 captured.
Review 28 captured.
Review 29 captured.
Review 30 captured.
Review 31 captured.
Review 32 captured.
Review 33 captured.
Review 34 captured.
Review 35 captured.
Review 36 captured.
Review 37 captured.
Review 38 captured.
Review 39 captured.
Review 40 captured.
Review 41 captured.
Review 42 captured.
Review 43 captured.
Review 44 captured.
Review 45 captured.
Review 46 captured.
Review 47 captured.
Review 48 captured.
Review 49 captured.
Review 50 captured.
Review 51 captured.
Review 52 captured.
Review 53 captured.
Review 54 captured.
Review 55 captured.
Review 56 captured.
Rev

Review 95 captured.
Review 96 captured.
Review 97 captured.
Review 98 captured.
Review 99 captured.
Review 100 captured.
Total reviews captured from this listing: 100.
Total reviews for vernon-tx so far: 956.


KeyboardInterrupt: 

In [125]:
url_list

['https://www.airbnb.com/s/newyork/homes',
 'https://www.airbnb.com/s/losangeles/homes',
 'https://www.airbnb.com/s/chicago/homes',
 'https://www.airbnb.com/s/dallas/homes',
 'https://www.airbnb.com/s/houston/homes',
 'https://www.airbnb.com/s/washington/homes',
 'https://www.airbnb.com/s/philadelphia/homes',
 'https://www.airbnb.com/s/miami/homes',
 'https://www.airbnb.com/s/atlanta/homes',
 'https://www.airbnb.com/s/boston/homes',
 'https://www.airbnb.com/s/phoenix/homes',
 'https://www.airbnb.com/s/sanfrancisco/homes',
 'https://www.airbnb.com/s/riverside/homes',
 'https://www.airbnb.com/s/detroit/homes',
 'https://www.airbnb.com/s/seattle/homes',
 'https://www.airbnb.com/s/minneapolis/homes',
 'https://www.airbnb.com/s/sandiego/homes',
 'https://www.airbnb.com/s/tampa/homes',
 'https://www.airbnb.com/s/denver/homes',
 'https://www.airbnb.com/s/baltimore/homes',
 'https://www.airbnb.com/s/st.louis/homes',
 'https://www.airbnb.com/s/orlando/homes',
 'https://www.airbnb.com/s/charlott